# Summarize an URL and give it in text and speech file

# TODO: robots.txt


Note: Web scraping might be against the terms of service of some websites. Make sure you have permission to scrape a website and always respect robots.txt and other rules that a website may have around web scraping.


pip install gtts
pip install sumy
pip install gensim
pip install beautifulsoup4
pip install requests

In [4]:
# Text to speech

from gtts import gTTS
import IPython.display as ipd

# The text you want to convert to speech
text = "Ferdowsi's Shahnameh,\
due to its special and unique place in the literary and  \
cultural history of Iran, has always been present  \
and influential in various fields and fields of life and thought of Iranians."

# Language in which you want to convert
# language = 'en'
# American accent
language = 'en-us' 

# Passing the text and language to the engine, slow=False makes the speech faster
tts = gTTS(text=text, lang=language, slow=False)

# Saving the converted audio in a mp3 file named 'welcome'
tts.save("welcome.mp3")

# Play the saved file
ipd.Audio("welcome.mp3", autoplay=True)


In [21]:

# URL of the article you want to summarize
# https://en.wikipedia.org/wiki/Shiraz"
# url = input('Inter your URL')
url = " https://en.wikipedia.org/wiki/Shiraz"

# NLTK (Natural Language Toolkit) is a
Python package for natural language processing. It provides a set of tools and resources for working with human language data, including tokenization, stemming, tagging, parsing, and semantic reasoning.

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
# summarize an URL and convert to text

from bs4 import BeautifulSoup
import requests
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

def fetch_data_from_url(url):
    # Fetch the content from the URL
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Assuming the article text is inside <p> HTML tags
    article_text = ' '.join([p.text for p in soup.find_all('p')])

    return article_text

def summarize_text(text, sentence_count=5):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, sentence_count)

    return ' '.join(str(sentence) for sentence in summary)


# Fetch and print the summarized data
article_text = fetch_data_from_url(url)
print("Original Article:")
print(article_text)

summary = summarize_text(article_text)
print("\n\nSummarized Article:")
print(summary)


In [22]:
# Get an URL and summarize and convert to text and then to mp3 file

from bs4 import BeautifulSoup
import requests
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from gtts import gTTS
import IPython.display as ipd

url = input('Inter your URL')

def fetch_data_from_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    #The soup.find_all('p') method is used to find all the <p> tags in the HTML and return them as a list.
    #See libs_needed file in this directory
    article_text = ' '.join([p.text for p in soup.find_all('p')])
    return article_text

# 5 means allow to the function to generate a summary with a specific length, in this case, 5 sentences.
def summarize_text(text, sentence_count=5):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, sentence_count)
    return [str(sentence) for sentence in summary]


def text_to_speech(my_text):
    my_language = 'en'
    tts = gTTS(text=my_text, lang=my_language, slow=False)
    tts.save("summary.mp3")
    return ipd.Audio("summary.mp3", autoplay=True) # when running,it is False ,why?

# Fetch, summarize, and read aloud
article_text = fetch_data_from_url(url)
summary_sentences = summarize_text(article_text)
print("Summarized Article:")
for sentence in summary_sentences:
    print(sentence)

text_to_speech(" ".join(summary_sentences))

Summarized Article:
[28] Shiraz soon returned to prosperity under the rule of Karim Khan Zand, who made it his capital in 1762.
Lacking any great industrial, religious or strategic importance, Shiraz became an administrative center, although its population has nevertheless grown considerably since the 1979 revolution.
[citation needed] Although some measures have been taken by the Municipality to preserve these gardens, many illegal developments still endanger them.
Iranian long-distance merchants from Fars developed marketing networks for these commodities, establishing trading houses in Bombay, Calcutta, Port Said, Istanbul and even Hong Kong.
Much older is the august Madrasa-e-Khan, or Khan Theological School, with about 600 students; its tile-covered buildings date from 1627.
